In [1]:
import pandas as pd
import random 
import numpy as np

## Data Cleaning

In [2]:
set3 = pd.read_csv('subset-3-sous-ensemble-3.csv', encoding='ISO-8859-1')
# print(set3[set3.LEVEL2ID!=0])
descrip = list(set3.DESCRIP_E.unique())
set3.drop(columns=['LEVEL2ID', 'LEVEL3ID','LEVEL4ID','LEVEL5ID','DESCRIP_F','TITLE_F','DEPT_F', 'INDICATORFRA','SUBINDICATORFRA'],inplace=True)
set3.head()

,LEVEL1ID,SURVEYR,BYCOND,DESCRIP_E,DEMCODE,QUESTION,TITLE_E,ANSWER1,ANSWER2,ANSWER3,...,MOST_NEGATIVE_OR_LEAST_POSITIVE,AGREE,SCORE5,SCORE100,ANSCOUNT,DEPT_E,INDICATORID,INDICATORENG,SUBINDICATORID,SUBINDICATORENG
0,0,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",35,45,6,...,14,81,3.99,75,73497,Public Service,4,WORKPLACE,14,Physical environment and equipment
1,1,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",41,45,5,...,10,85,4.14,79,11550,Canada Revenue Agency,4,WORKPLACE,14,Physical environment and equipment
2,2,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",45,42,4,...,9,87,4.21,80,5626,Employment and Social Development Canada,4,WORKPLACE,14,Physical environment and equipment
3,3,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",29,48,6,...,16,78,3.86,72,7308,Department of National Defence,4,WORKPLACE,14,Physical environment and equipment
4,4,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",20,46,10,...,23,67,3.58,64,3252,Correctional Service Canada,4,WORKPLACE,14,Physical environment and equipment


In [3]:
# There are 59 equity groups
len(set3.DESCRIP_E.unique())
list(set3.DESCRIP_E.unique())

['Male gender',
 'Female gender',
 'Gender diverse',
 'Indigenous',
 'Non-Indigenous',
 'First Nation (North American Indian)',
 'Métis',
 'Inuk (Inuit)',
 'Person with a disability',
 'Not a person with a disability',
 'A seeing disability affects vision, including total blindness, partial sight and visual distortion',
 'A seeing disability affects vision, including total blindness, partial sight and visual distortion - Not selected',
 'A hearing disability affects ability to hear, including being hard of hearing, deafness or acoustic distortion',
 'A hearing disability affects ability to hear, including being hard of hearing, deafness or acoustic distortion - Not selected',
 'A mobility issue affects ability to move your body, including the required use of a wheelchair or a cane, or other issues impacting your mobility',
 'A mobility issue affects ability to move your body, including the required use of a wheelchair or a cane, or other issues impacting your mobility - Not selected',


## Delete rows that have no survey data (more than 50%)
- bc some dept don't have certain equity groups

In [4]:
print(len(set3[set3['ANSWER1']== ' '].index))
print(len(set3[set3['ANSWER5']== ' '].index))
print(len(set3[set3['ANSWER2']!= ' '].index))

# clean out empty rows
print(len(set3))
set3 = set3[set3['ANSWER2']!= ' ']
set3.reset_index(inplace=True, drop=True)
# set3_2020.to_csv('subset3_2020_cleaned.csv')
set3.head()

822456
822456
583123
1405579


,LEVEL1ID,SURVEYR,BYCOND,DESCRIP_E,DEMCODE,QUESTION,TITLE_E,ANSWER1,ANSWER2,ANSWER3,...,MOST_NEGATIVE_OR_LEAST_POSITIVE,AGREE,SCORE5,SCORE100,ANSCOUNT,DEPT_E,INDICATORID,INDICATORENG,SUBINDICATORID,SUBINDICATORENG
0,0,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",35,45,6,...,14,81,3.99,75,73497,Public Service,4,WORKPLACE,14,Physical environment and equipment
1,1,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",41,45,5,...,10,85,4.14,79,11550,Canada Revenue Agency,4,WORKPLACE,14,Physical environment and equipment
2,2,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",45,42,4,...,9,87,4.21,80,5626,Employment and Social Development Canada,4,WORKPLACE,14,Physical environment and equipment
3,3,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",29,48,6,...,16,78,3.86,72,7308,Department of National Defence,4,WORKPLACE,14,Physical environment and equipment
4,4,2020,Q115 = 1,Male gender,2011,Q01,"Question 1. I have the tools, technology and e...",20,46,10,...,23,67,3.58,64,3252,Correctional Service Canada,4,WORKPLACE,14,Physical environment and equipment


## Take year 2020 data - majority is 2020 yr data
- 2020 has 330k rows
- 2019 has 176k rows
- 2018 has 76k rows

In [5]:
print(set3.columns)
print(set3.SURVEYR.unique())
print(len(set3[set3.SURVEYR==2020]))

# have majority of data form 2020 so we focus on this year
set3_2020 = set3[set3.SURVEYR==2020]

Index(['LEVEL1ID', 'SURVEYR', 'BYCOND', 'DESCRIP_E', 'DEMCODE', 'QUESTION',
       'TITLE_E', 'ANSWER1', 'ANSWER2', 'ANSWER3', 'ANSWER4', 'ANSWER5',
       'ANSWER6', 'ANSWER7', 'MOST_POSITIVE_OR_LEAST_NEGATIVE',
       'NEUTRAL_OR_MIDDLE_CATEGORY', 'MOST_NEGATIVE_OR_LEAST_POSITIVE',
       'AGREE', 'SCORE5', 'SCORE100', 'ANSCOUNT', 'DEPT_E', 'INDICATORID',
       'INDICATORENG', 'SUBINDICATORID', 'SUBINDICATORENG'],
      dtype='object')
[2020 2019 2018]
330739


### number of equity groups in all 68 departments
- min = 2 (some department only have male & female?)
- max = 59 (all equity groups in the survey)

In [6]:
# dept_series = pd.Series(set3_2020.groupby('DEPT_E').size())
# for i in range(len(dept_series)):
#     print(dept_series[i], '\t', list(dept_series.index)[i])
# dept_series
set3_2020['DEPT_E'].unique()
print('by # of equity groups surveyedin departments')
dept_equity = pd.Series(dtype=int)
for dept in set3_2020['DEPT_E'].unique():
    num_equity_groups = len(set3_2020[set3_2020['DEPT_E']==dept].DESCRIP_E.unique())
    dept_equity[dept] = num_equity_groups
dept_equity.describe()

by # of equity groups surveyedin departments


count    68.000000
mean     37.720588
std      16.127149
min       2.000000
25%      25.000000
50%      38.500000
75%      53.000000
max      59.000000
dtype: float64

### Look at the department that has only 2 equity groups
- Looks like it's just female & male
- We can't assume other things, but only the gender of surveyed people in that department
- but how is that possible when "Not Selected" is also an attribute value   -_-|||
- Gonna ignore the "not selected" groups

In [7]:
dept_2 = dept_equity[dept_equity == 2].index[0]
print(dept_2)
set3_2020[set3_2020['DEPT_E']==dept_2]['DESCRIP_E'].unique()

Canadian Security Intelligence Service


array(['Male gender', 'Female gender'], dtype=object)

### Look at how many equity group has 'Not Selected'
- Let's see if there's rows selected **"... - Not selected"**

In [8]:
descrip = list(set3.DESCRIP_E.unique())
selected = []
not_selected = []
for group in descrip:
    if ("Not selected" in group):
        not_selected.append(group)
    else:
        selected.append(group)
        
print(f'Not selected - {len(not_selected)}')
print(f'Selected - {len(selected)}')
print()
num_not_Chinese = len(set3_2020[set3_2020['DESCRIP_E']=='Chinese - Not selected'])
print(f"Is there rows with '...- Not selected'? -- YES \n{num_not_Chinese} rows with 'Chinese - Not selected'", )
not_selected


Not selected - 21
Selected - 38

Is there rows with '...- Not selected'? -- YES 
6276 rows with 'Chinese - Not selected'


['A seeing disability affects vision, including total blindness, partial sight and visual distortion - Not selected',
 'A hearing disability affects ability to hear, including being hard of hearing, deafness or acoustic distortion - Not selected',
 'A mobility issue affects ability to move your body, including the required use of a wheelchair or a cane, or other issues impacting your mobility - Not selected',
 'An issue with flexibility or dexterity affects ability to move joints or perform motor tasks, especially with your hands - Not selected',
 'A mental health issue affects psychology or behaviour, such as anxiety, depression or social / compulsive disorder or phobia or psychiatric illness - Not selected',
 'A sensory / environmental disability affects sensitivity to light, sounds or other distractions, as well as allergens and other environmental sensitivities - Not selected',
 "A chronic health condition or pain affects ability to function on a regular or episodic basis due to mi

In [9]:
additional_selected = []
for i in selected:
    included = False
    for j in not_selected:
        if i in j:
            included = True
    if not included:
        additional_selected.append(i)
len(additional_selected)
additional_selected

['Male gender',
 'Female gender',
 'Gender diverse',
 'Non-Indigenous',
 'First Nation (North American Indian)',
 'Métis',
 'Inuk (Inuit)',
 'Person with a disability',
 'Not a person with a disability',
 'Visible minority',
 'Non-visible minority',
 'Heterosexual',
 'Gay or lesbian',
 'Bisexual',
 'Another sexual orientation',
 'Prefer not to answer']

In [10]:
group_compare = set3_2020.drop(by=['MOST_NEGATIVE_OR_LEAST_POSITIVE', 'MOST_POSITIVE_OR_LEAST_NEGATIVE', 'NEUTRAL_OR_MIDDLE_CATEGORY'])

In [19]:
print(set3.columns)

# set3_2020[set3_2020['INDICATORENG','SUBINDICATORENG'] == set3_2020[['INDICATORENG','SUBINDICATORENG']]]
set3_2020[['TITLE_E','INDICATORENG','SUBINDICATORENG']]


Index(['LEVEL1ID', 'SURVEYR', 'BYCOND', 'DESCRIP_E', 'DEMCODE', 'QUESTION',
       'TITLE_E', 'ANSWER1', 'ANSWER2', 'ANSWER3', 'ANSWER4', 'ANSWER5',
       'ANSWER6', 'ANSWER7', 'MOST_POSITIVE_OR_LEAST_NEGATIVE',
       'NEUTRAL_OR_MIDDLE_CATEGORY', 'MOST_NEGATIVE_OR_LEAST_POSITIVE',
       'AGREE', 'SCORE5', 'SCORE100', 'ANSCOUNT', 'DEPT_E', 'INDICATORID',
       'INDICATORENG', 'SUBINDICATORID', 'SUBINDICATORENG'],
      dtype='object')


,TITLE_E,INDICATORENG,SUBINDICATORENG
0,"Question 1. I have the tools, technology and e...",WORKPLACE,Physical environment and equipment
1,"Question 1. I have the tools, technology and e...",WORKPLACE,Physical environment and equipment
2,"Question 1. I have the tools, technology and e...",WORKPLACE,Physical environment and equipment
3,"Question 1. I have the tools, technology and e...",WORKPLACE,Physical environment and equipment
4,"Question 1. I have the tools, technology and e...",WORKPLACE,Physical environment and equipment
...,...,...,...
582992,Question 97. To what extent have issues with t...,COMPENSATION,Pay or other compensation issues
582993,Question 97. To what extent have issues with t...,COMPENSATION,Pay or other compensation issues
582994,Question 97. To what extent have issues with t...,COMPENSATION,Pay or other compensation issues
582995,Question 97. To what extent have issues with t...,COMPENSATION,Pay or other compensation issues
